In [10]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"adityajn105","key":"74ac3c08ead73b4bb100dc52c557f763"}'}

In [13]:
!mkdir /root/.kaggle
!mv kaggle.json /root/.kaggle/
!kaggle competitions download -c aerial-cactus-identification
!unzip -q train.zip
!unzip -q test.zip
!rm train.zip test.zip

train.csv: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
  0% 0.00/4.20M [00:00<?, ?B/s]
100% 4.20M/4.20M [00:00<00:00, 67.7MB/s]
 62% 12.0M/19.2M [00:00<00:00, 124MB/s]
100% 19.2M/19.2M [00:00<00:00, 123MB/s]
replace train/008bd3d84a1145e154409c124de7cee9.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace train/0090d921aeb53be7e3df6f4b0254c537.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
replace test/000940378805c44108d287872b2f04ce.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [14]:
import glob
import pandas as pd
import numpy as np
data = pd.read_csv('train.csv')
data['path'] = 'train/'
data['path'] = data[['path','id']].apply(lambda x: "".join(x), axis=1)
data.drop(['id'],axis=1,inplace=True)
data.head()

,has_cactus,path
0,1,train/0004be2cfeaba1c0361d39e2b000257b.jpg
1,1,train/000c8a36845c0208e833c79c1bffedd1.jpg
2,1,train/000d1e9a533f62e55c289303b072733d.jpg
3,1,train/0011485b40695e9138e92d0b3fb55128.jpg
4,1,train/0014d7a11e90b62848904c1418fc8cf2.jpg


In [15]:
all_images = glob.glob('train/*')
print(len(all_images), data.shape)

IMG_DIM = (30,30,3)

from keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
imgs = np.array([ img_to_array( load_img(file, target_size = IMG_DIM) ) for file in data.path.values ])
labels = data.has_cactus.values


print(imgs.shape,labels.shape)

17500 (17500, 2)
(17500, 30, 30, 3) (17500,)


In [0]:
from sklearn.model_selection import train_test_split
X_train,X_val,Y_train,Y_val = train_test_split(imgs,labels,test_size=0.3, stratify=labels)

train_datagen = ImageDataGenerator(rescale=1./255, 
                                   zoom_range=0.3, 
                                   rotation_range=50,
                                   width_shift_range=0.2, 
                                   height_shift_range=0.2, 
                                   shear_range=0.2, 
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow(X_train, Y_train, batch_size=32)
val_generator = val_datagen.flow(X_val, Y_val, batch_size=32)

In [17]:
from keras.layers import Input,Conv2D,Dense,Dropout, MaxPooling2D, Flatten
from keras import optimizers
from keras.models import Model

#Input Layer
inp = Input(IMG_DIM)

#1st Conv
conv_1  = Conv2D( 64, kernel_size=(2,2), activation='relu')(inp)
pool_1 = MaxPooling2D(pool_size=(2,2))(conv_1)

#2nd Conv
conv_2 = Conv2D( 32, kernel_size=(2,2), activation='relu')(pool_1)
pool_2 = MaxPooling2D(pool_size=(2,2))(conv_2)

#flatten
flatten = Flatten()(pool_2)
dropout_1 = Dropout(0.3)(flatten)

#1st Dense
dense_1 = Dense(512, activation='relu')(dropout_1)
dropout_2 = Dropout(0.3)(dense_1)

#2nd Dense
dense_2 = Dense(64,activation='relu')(dropout_2)
dropout_3 = Dropout(0.2)(dense_2)

#output
output = Dense(1, activation='sigmoid')(dropout_3)

model = Model(inp,output)

model.compile( loss='binary_crossentropy', optimizer=optimizers.Adam(lr=1e-4), metrics=['accuracy'] )

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 30, 30, 3)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 29, 29, 64)        832       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 32)        8224      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1152)              0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 1152)              0         
__________

In [19]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

model_checkpoint  = ModelCheckpoint('model_best_checkpoint.h5', save_best_only=True, monitor='val_acc', mode='max', verbose=2)
early_stopping = EarlyStopping(monitor='val_loss', patience=15, mode='min')

callback_list = [model_checkpoint]

history = model.fit_generator(train_generator, steps_per_epoch=100, epochs=300,
                              validation_data=val_generator, validation_steps=50, 
                              verbose=2,callbacks=callback_list)

Epoch 1/300
 - 2s - loss: 0.2003 - acc: 0.9249 - val_loss: 0.2973 - val_acc: 0.8987

Epoch 00001: val_acc improved from -inf to 0.89873, saving model to model_best_checkpoint.h5
Epoch 2/300
 - 2s - loss: 0.1998 - acc: 0.9237 - val_loss: 0.1619 - val_acc: 0.9406

Epoch 00002: val_acc improved from 0.89873 to 0.94063, saving model to model_best_checkpoint.h5
Epoch 3/300
 - 2s - loss: 0.1818 - acc: 0.9378 - val_loss: 0.1967 - val_acc: 0.9313

Epoch 00003: val_acc did not improve from 0.94063
Epoch 4/300
 - 2s - loss: 0.1967 - acc: 0.9231 - val_loss: 0.2761 - val_acc: 0.8975

Epoch 00004: val_acc did not improve from 0.94063
Epoch 5/300
 - 2s - loss: 0.1847 - acc: 0.9310 - val_loss: 0.2605 - val_acc: 0.8981

Epoch 00005: val_acc did not improve from 0.94063
Epoch 6/300
 - 2s - loss: 0.1957 - acc: 0.9269 - val_loss: 0.2473 - val_acc: 0.9087

Epoch 00006: val_acc did not improve from 0.94063
Epoch 7/300
 - 2s - loss: 0.2044 - acc: 0.9200 - val_loss: 0.1643 - val_acc: 0.9375

Epoch 00007: val

In [20]:
tests =  glob.glob('test/*')
imgs = np.array([ img_to_array( load_img(file, target_size = IMG_DIM) ) for file in tests ])/255
submission = pd.DataFrame({'id':tests})
submission.id = submission.id.apply(lambda x: x.split('/')[1])
submission.head()

,id
0,727513b9a1f61e2dd1a416d4fc257e60.jpg
1,e0251fcc9539ecaac51bd1794b1fdff5.jpg
2,bc71ed087b1f7086e49bf0e8773b514f.jpg
3,3e3c941e0a06a74e3b012d31b4abc34e.jpg
4,9f6bd8a79a7179d3f725b6f477f3747f.jpg


In [22]:
submission['has_cactus'] = np.squeeze(model.predict(imgs))
submission.to_csv('submission_cactus.csv',index=False)
submission.head()

,id,has_cactus
0,727513b9a1f61e2dd1a416d4fc257e60.jpg,0.999976
1,e0251fcc9539ecaac51bd1794b1fdff5.jpg,0.999989
2,bc71ed087b1f7086e49bf0e8773b514f.jpg,1.000000
3,3e3c941e0a06a74e3b012d31b4abc34e.jpg,0.999982
4,9f6bd8a79a7179d3f725b6f477f3747f.jpg,0.999980


In [0]:
from google.colab import files
files.download('submission_cactus.csv')
